# Exemplos Cutting Planes

**Preâmbulo**

In [49]:
%load_ext autoreload
%autoreload 2
%pip install numpy scipy
import numpy as np
from gommory_cuts import cutting_planes, _non_integer_mask
from scipy.optimize import milp, LinearConstraint

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Exemplo 5.12

In [76]:
A = np.array([
    [ 2, -1,  1, -2, 1, 0,  0,  0,  0],
    [-1,  2, -1,  1, 0, 1,  0,  0,  0],
    [ 2,  1, -1,  0, 0, 0, -1,  0,  0],
    [ 0,  1,  0,  0, 0, 0,  0, -1,  0],
    [ 0,  0,  0,  1, 0, 0,  0,  0, -1]
])
b = np.array([
    6,
    8,
    2,
    1,
    2
])
c = np.array([
    1, 2, 3, -1, 0, 0, 0, 0, 0
])

m, n = A.shape

I = list(
    range(n - m,  n)
)

z_star, x_star, I_star, iters, solution_type, steps = cutting_planes(A, b, c, I)
z_star, x_star

(np.float64(-4.0),
 array([4.092e+03, 1.000e+00, 0.000e+00, 4.098e+03, 1.900e+01, 0.000e+00,
        8.183e+03, 0.000e+00, 4.096e+03, 1.000e+00, 0.000e+00]))

**Comparação com o Scipy**

In [77]:
constraints = LinearConstraint(A, b, b)
integrality = np.array([1] * (n - m) + [0] * m)
res = milp(
    c = c,
    constraints = constraints,
    integrality = integrality
)

first_condition = np.isclose(z_star, res.fun) # z_star == z solver
first_condition &= np.all(~_non_integer_mask(x_star[0:(n - m)])) # x_star should be integer at the expected integer indices
second_condition = np.all(A @ x_star[0:9] <= b) # All x_star without artificial variables feasible
second_condition &= z_star <= res.fun # z_star better or equal solver
second_condition &= np.all(~_non_integer_mask(x_star[0:(n - m)])) # x_star should be integer at the expected integer indices

assert first_condition or second_condition
print(f'Results from cutting plane method:\nz_star, x_star\n{z_star, x_star}\n')
print(f'Results from scipy milp solver:\nz_star, x_star\n{res.fun, res.x}\n')
print(f'Gap ((z_milp - z_cp)/z_milp): {(res.fun - z_star)/res.fun}')

Results from cutting plane method:
z_star, x_star
(np.float64(-4.0), array([4.092e+03, 1.000e+00, 0.000e+00, 4.098e+03, 1.900e+01, 0.000e+00,
       8.183e+03, 0.000e+00, 4.096e+03, 1.000e+00, 0.000e+00]))

Results from scipy milp solver:
z_star, x_star
(-3.0, array([ 1.,  1.,  0.,  6., 17.,  1.,  1.,  0.,  4.]))

Gap ((z_milp - z_cp)/z_milp): -0.3333333333333333


## Exemplo 5.13

In [78]:
A = np.array([
    [ 3, 1, 1, 1, 0, 0],
    [-1, 1, 0, 0, 1, 0],
    [ 0, 1, 2, 0, 0, 1]
])
b = np.array([
    12,
    4,
    8
])
c = np.array([-2, -1, -3, 0, 0, 0])
m, n = A.shape
I = list(
    range(n - m, n)
)
z_star, x_star, I_star, iters, solution_type, steps = cutting_planes(A, b, c, I)
print(z_star, x_star)

-15.999999999999396 [2.00000000e+00 0.00000000e+00 4.00000000e+00 2.00000000e+00
 6.00000000e+00 0.00000000e+00 1.22820000e+04 1.00000000e+00
 1.66533454e-16 3.88578059e-16 0.00000000e+00]


**Comparação com o scipy**

In [80]:
constraints = LinearConstraint(A, b, b)
integrality = np.array([1] * (n - m) + [0] * m)
res = milp(
    c = c,
    constraints = constraints,
    integrality = integrality
)

first_condition = np.isclose(z_star, res.fun) # z_star == z solver
first_condition &= np.all(~_non_integer_mask(x_star[0:(n - m)])) # x_star should be integer at the expected integer indices
second_condition = np.all(A @ x_star[0:6] <= b) # All x_star without artificial variables feasible
second_condition &= z_star <= res.fun # z_star better or equal solver
second_condition &= np.all(~_non_integer_mask(x_star[0:(n - m)])) # x_star should be integer at the expected integer indices

assert first_condition or second_condition
print(f'Results from cutting plane method:\nz_star, x_star\n{z_star, x_star}\n')
print(f'Results from scipy milp solver:\nz_star, x_star\n{res.fun, res.x}\n')
print(f'Gap ((z_milp - z_cp)/z_milp): {(res.fun - z_star)/res.fun}')

Results from cutting plane method:
z_star, x_star
(np.float64(-15.999999999999396), array([2.00000000e+00, 0.00000000e+00, 4.00000000e+00, 2.00000000e+00,
       6.00000000e+00, 0.00000000e+00, 1.22820000e+04, 1.00000000e+00,
       1.66533454e-16, 3.88578059e-16, 0.00000000e+00]))

Results from scipy milp solver:
z_star, x_star
(-16.0, array([2., 0., 4., 2., 6., 0.]))

Gap ((z_milp - z_cp)/z_milp): 3.774758283725532e-14


## Exemplo 5.14

In [82]:
A = np.array([
    [1, 3,  1, 1,  0],
    [2, 1, -1, 0, -1]
])
b = np.array([
    8,
    3
])
c = np.array([-2, -3, 2, 0, 0])
m, n = A.shape
I = list(
    range(n - m, n)
)
z_star, x_star, I_star, iters, solution_type, steps = cutting_planes(A, b, c, I)
print(z_star, x_star)

-16.0 [8.000e+00 0.000e+00 0.000e+00 0.000e+00 1.300e+01 8.184e+03]


**Comparação com o scipy**

In [83]:
constraints = LinearConstraint(A, b, b)
integrality = np.array([1] * (n - m) + [0] * m)
res = milp(
    c = c,
    constraints = constraints,
    integrality = integrality
)

first_condition = np.isclose(z_star, res.fun) # z_star == z solver
first_condition &= np.all(~_non_integer_mask(x_star[0:(n - m)])) # x_star should be integer at the expected integer indices
second_condition = np.all(A @ x_star[0:5] <= b) # All x_star without artificial variables feasible
second_condition &= z_star <= res.fun # z_star better or equal solver
second_condition &= np.all(~_non_integer_mask(x_star[0:(n - m)])) # x_star should be integer at the expected integer indices

assert first_condition or second_condition
print(f'Results from cutting plane method:\nz_star, x_star\n{z_star, x_star}\n')
print(f'Results from scipy milp solver:\nz_star, x_star\n{res.fun, res.x}\n')
print(f'Gap ((z_milp - z_cp)/z_milp): {(res.fun - z_star)/res.fun}')

Results from cutting plane method:
z_star, x_star
(np.float64(-16.0), array([8.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.300e+01, 8.184e+03]))

Results from scipy milp solver:
z_star, x_star
(-16.0, array([ 8., -0.,  0.,  0., 13.]))

Gap ((z_milp - z_cp)/z_milp): -0.0
